<a href="https://colab.research.google.com/github/drshahizan/special-topic-data-engineering/blob/main/project/submission%20/Gadgeteen/Gadgeteen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Retrieving


In [1]:
CLIENT_ID = '-R0ZxQ_ODZQhIXv0dWtWZw'
SECRET_KEY = 'NZFugdkW8vcg0dBMIadBMruyejgKkA'

In [2]:
import requests
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [3]:
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)
data = {
    'grant_type' : 'password',
    'username' : 'gadgeteen',
    'password' : 'gadgetech'
}
headers = {'User-Agent' : 'MyAPI/0.0.1'}
res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
TOKEN = res.json()['access_token']

In [4]:
headers['Authorization'] = f'bearer {TOKEN}'
params = {
    'limit': 100,
  }

data = []
for i in range(5):
  count = i * 100

  params['count'] = count

  res = requests.get("https://oauth.reddit.com/r/ChatGPT/new",headers=headers, params=params)
  for post in res.json()['data']['children']:
    post_data = {
          'subreddit': post['data']['subreddit'],
          'title': post['data']['title'],
          'selftext': post['data']['selftext'],
          'upvote_ratio': post['data']['upvote_ratio'],
          'ups': post['data']['ups'],
          'downs': post['data']['downs'],
          'score': post['data']['score']
    }
    name = post['data']['name']
    data.append(post_data)
  params['after'] = name
df = pd.DataFrame(data)
df.shape

(500, 7)

# Data Cleaning and Preprocessing

In [5]:
import nltk
import re
from nltk.corpus import stopwords

# Download the necessary NLTK resources (run only once)
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
# Drop any duplicate rows
df.drop_duplicates(inplace=True)

# Remove any missing values (NaN)
df.dropna(inplace=True)

# Remove leading/trailing whitespaces in the text columns
df['title'] = df['title'].str.strip()
df['selftext'] = df['selftext'].str.strip()

# Convert the 'title' and 'selftext' columns to lowercase and remove punctuation
df['clean_text'] = df['title'].str.lower() + ' ' + df['selftext'].str.lower()
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Reset the index
df.reset_index(drop=True, inplace=True)

In [7]:
# Customized list of stopwords
custom_stopwords = set(stopwords.words('english'))
custom_stopwords.update(['chatgpt'])

# Function to process text and remove stopwords
def process_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in custom_stopwords]
    return tokens

# Apply text processing and generate word frequency
df['processed_text'] = df['clean_text'].apply(process_text)
df['joined_text'] = df['processed_text'].apply(lambda tokens: ' '.join(tokens))

columns_to_keep = ['subreddit', 'title', 'selftext', 'upvote_ratio', 'ups', 'downs', 'score', 'clean_text', 'processed_text', 'joined_text']
df = df[columns_to_keep]

# Sentiment Analysis

In [9]:
from nltk.sentiment import SentimentIntensityAnalyzer
from collections import Counter

# Initialize the Sentiment Intensity Analyzer
sia = SentimentIntensityAnalyzer()

# Apply sentiment analysis to the 'title' column
df['sentiment'] = df['joined_text'].apply(lambda x: sia.polarity_scores(x)['compound'])
df['csentiment'] = df['clean_text'].apply(lambda x: sia.polarity_scores(x)['compound'])
# Classify the sentiment
threshold = 0.1

df['sentiment_class'] = df['sentiment'].apply(lambda x: 'positive' if x >= threshold else 'negative' if x <= -threshold else 'neutral')

df[['clean_text', 'sentiment', 'csentiment', 'sentiment_class']]

,clean_text,sentiment,csentiment,sentiment_class
0,tutorial how to change bings hidden settings did you know most of the settings for bing like the conversation limits if you have image input etc are actually stored locally this means you can change these settings for yourself through our good friend devtools\n\nheres how to find them 1 go to bing chat at httpsbingcomchathttpsbingcomchat\n\n2 press ctrl shift i to open devtools\n\n3 go to the sources tab\n\n4 go here\n\nhttpspreviewreddit3rshm9dykfbb1pngwidth923ampformatpngampautowebpampsbb30050f5a054138962e064b003d6413b3474e12\n\n5 press ctrl f and search for sydconvconfig and you will find where the settings are\n\nyou cannot edit them here directly so we will have to use the console\n\nin order for the page to act as normal i recommend copying everything from the _w highlighted in blue in the image all the way to here\n\nhttpspreviewreddityon4qo70lfbb1pngwidth913ampformatpngampautowebpamps805ec92d84aabf95171de6df954890386c21aff7\n\n6 after you select that all go over to the console tab and paste it all\n\n7 you can now change whatever setting you feel like here for the conversation limit and image input find maxturnsperconversation and enablevisualsearch for yesno settings 0 no and 1 yes\n\n8 do not press enter your changes wont be saved once your done changing your settings select all with ctrl a and copy\n\n9 heres the trickiest step you have to reload the page with ctrl r paste the settings and hit enter before the whole page loads try to be as quick as possible\n\n10 if it works congratulations welcome to your bing,0.9563,0.9474,positive
1,teaching gpt riddles i was curious if it could answer riddles so i decided to ask it one from my childhood the second answer it gave really through me for a loop but i decided to see if i could get it to not only learn the answer but understand the answer im pretty happy with how it turned out,0.8481,0.9001,positive
2,ah yes tails is a robot,0.4019,0.4019,positive
3,3 new sports created by chatgpt phenomenon the lack of the invention of new sports\n\nexplanation the apparent lack of the invention of new sports may seem puzzling but this can be explained by several factors that converge from various fields of knowledge including sociology psychology economics and anthropology\n\ntradition and cultural momentum sports often emerge from specific cultural contexts and become ingrained in society over time they are passed down through generations and form part of a shared identity this cultural momentum makes it hard for new sports to gain acceptance\n\neconomic factors existing sports have established infrastructures industries and financial investments new sports would have to compete against these already established systems and would need significant capital for promotion infrastructure and talent development\n\ncognitive load and novelty management human beings though capable of adaptation and learning tend to favor familiarity and predictability which reduces cognitive load this tendency often extends to sports where understanding and appreciating the rules strategies and nuances of a new sport can be challenging\n\nphysical constraints human physical capabilities and limitations also shape sports most sports have already explored and optimized different areas of physical abilities strength agility endurance coordination etc making it difficult to invent entirely new sports that dont closely resemble existing ones\n\ngiven these factors the invention of new sports is challenging but not impossible the key is to create a sport that can leverage existing infrastructures provides an exciting twist to known rules or skills and taps into unexplored areas of human capabilities\n\nexamples of new sports\n\naerodisc this is a 3team sport with each team consisting of 5 players the game is played in a large spherical arena with three equidistant goals the objective is to throw a disc into the opponents goals while avoiding them scoring in your

In [10]:
# Aggregate sentiment scores to calculate overall sentiment
overall_sentiment = df['csentiment'].mean()
overall_join_sentiment = df['sentiment'].mean()

# Display the overall sentiment score
print("Overall Sentiment Score with stopword:", overall_sentiment)
print("Overall Joined Sentiment Score without stopword:", overall_join_sentiment)

Overall Sentiment Score with stopword: 0.26893039999999996
Overall Joined Sentiment Score without stopword: 0.30062419999999995


# Export Data

In [ ]:
from pymongo import MongoClient

# Establish a connection to your MongoDB server
client = MongoClient('mongodb://localhost:27017/')

# Access the desired database and collection
db = client.chatgpt_sentiment
collection = db.reddit_posts

# Convert your DataFrame to a list of dictionaries
data = df.to_dict(orient='records')

# Insert the data into the MongoDB collection
collection.insert_many(data)


In [8]:
# Save the cleaned data to a new CSV file
df.to_csv('cleaned_social_media_data.csv', index=False)
df

,subreddit,title,selftext,upvote_ratio,ups,downs,score,clean_text,processed_text,joined_text
0,ChatGPT,Tutorial: How to change Bing's hidden settings,"Did you know: most of the settings for Bing like the conversation limits, if you have image input, etc. are actually stored locally. This means you can change these settings for yourself through our good friend DevTools.\n\nHere's how to find them: 1. Go to Bing Chat at [https://bing.com/chat](https://bing.com/chat).\n\n2. Press Ctrl + Shift + I to open DevTools.\n\n3. Go to the Sources tab.\n\n4. Go here:\n\nhttps://preview.redd.it/3rshm9dykfbb1.png?width=923&amp;format=png&amp;auto=webp&amp;s=bb30050f5a054138962e064b003d6413b3474e12\n\n5. Press Ctrl + F and search for ""sydConvConfig"" and you will find where the settings are.\n\nYou cannot edit them here directly, so we will have to use the Console.\n\nIn order for the page to act as normal, I recommend copying everything from the ""\_w"" highlighted in blue in the image all the way to here:\n\nhttps://preview.redd.it/yon4qo70lfbb1.png?width=913&amp;format=png&amp;auto=webp&amp;s=805ec92d84aabf95171de6df954890386c21aff7\n\n6. After you select that all, go over to the Console tab and paste it all.\n\n7. You can now change whatever setting you feel like here. For the conversation limit and image input, find ""maxTurnsPerConversation"" and ""enableVisualSearch"". For yes/no settings, 0 = no and 1 = yes.\n\n8. DO NOT PRESS ENTER. Your changes won't be saved. Once your done changing your settings, select all with Ctrl + A and copy.\n\n9. Here's the trickiest step, you have to reload the page with Ctrl + R, paste the settings, and hit enter before the whole page loads. Try to be as quick as possible.\n\n10. If it works, congratulations! Welcome to \*your\* Bing.",1.00,3,0,3,tutorial how to change bings hidden settings did you know most of the settings for bing like the conversation limits if you have image input etc are actually stored locally this means you can change these settings for yourself through our good friend devtools\n\nheres how to find them 1 go to bing chat at httpsbingcomchathttpsbingcomchat\n\n2 press ctrl shift i to open devtools\n\n3 go to the sources tab\n\n4 go here\n\nhttpspreviewreddit3rshm9dykfbb1pngwidth923ampformatpngampautowebpampsbb30050f5a054138962e064b003d6413b3474e12\n\n5 press ctrl f and search for sydconvconfig and you will find where the settings are\n\nyou cannot edit them here directly so we will have to use the console\n\nin order for the page to act as normal i recommend copying everything from the _w highlighted in blue in the image all the way to here\n\nhttpspreviewreddityon4qo70lfbb1pngwidth913ampformatpngampautowebpamps805ec92d84aabf95171de6df954890386c21aff7\n\n6 after you select that all go over to the console tab and paste it all\n\n7 you can now change whatever setting you feel like here for the conversation limit and image input find maxturnsperconversation and enablevisualsearch for yesno settings 0 no and 1 yes\n\n8 do not press enter your changes wont be saved once your done changing your settings select all with ctrl a and copy\n\n9 heres the trickiest step you have to reload the page with ctrl r paste the settings and hit enter before the whole page loads try to be as quick as possible\n\n10 if it works congratulations welcome to your bing,"[tutorial, change, bings, hidden, settings, know, settings, bing, like, conversation, limits, image, input, etc, actually, stored, locally, means, change, settings, good, friend, devtools, heres, find, go, bing, chat, httpsbingcomchathttpsbingcomchat, press, ctrl, shift, open, devtools, go, sources, tab, go, press, ctrl, f, search, sydconvconfig, find, settings, edit, directly, use, console, order, page, act, normal, recommend, copying, everything, highlighted, blue, image, way, select, go, console, tab, paste, change, whatever, setting, feel, like, conversation, limit, image, input, find, maxturnsperconversation, enablevisualsearch, yesno, se